# Add Waveforms
Here we combine the experiment-specific stationXML file created in GLNN_StationXML.ipynb with the recorded AE waveforms from TranAX in a tpc5 file to initialize the experiment ASDF file.

ElSys has provided a module tpc5.py __[available on their website](https://www.elsys-instruments.com/en/support/tpc5_fileformat.php)__ for easy interpretation.

At the bottom, I also add the TDMS waveforms for my load transducers recorded with LabView.

In [18]:
import tpc5 # get from ElSys, see link above
import h5py
import numpy as np
import pyasdf
from obspy import Stream, Trace
from obspy.core.util import AttribDict
from obspy.core import Stats
import obspy
from nptdms import TdmsFile

First set up the ASDF dataset to fill

In [2]:
ds = pyasdf.ASDFDataSet('020918_shear/020918ASDF.h5')
# ds = pyasdf.ASDFDataSet('020918_shear/020918ASDF.h5', compression='gzip-3')
#ds.add_stationxml('020918_shear/GLNNstations_020918.xml')

In [3]:
ds.waveforms.list()

['L0.AE09',
 'L0.AE11',
 'L0.AE16',
 'L0.AE17',
 'L0.AE18',
 'L0.AE21',
 'L0.AE22',
 'L0.AE23',
 'L0.AE27',
 'L0.AE28',
 'L0.AE32',
 'L0.AE33',
 'L0.AE35',
 'L0.AE38',
 'L0.AE41',
 'L0.AE43']

In [4]:
ds.waveforms.L0_AE09.StationXML.networks[0].stations[0]
# ASDF appears to dump the extra attribute of the StationXML
# Add the (x,y) locations as auxiliary data instead

Station AE09 (BP3)
	Station Code: AE09
	Channel Count: 1/None (Selected/Total)
	None - 
	Access: None 
	Latitude: 37.87, Longitude: -122.26, Elevation: 100.0 m
	Available Channels:
		AE09.00.FHZ

The stations are ordered as A1-D4 in the original StationXML but get re-sorted to increasing station codes when imported to ASDF. The ASDF format also drops the .extra attribute from the StationXML, which contained my local_location information. I need to temporarily access the StationXML outside of ASDF to bring back this lost information.

In [5]:
inv = obspy.read_inventory('020918_shear/GLNNstations_020918.xml', format='stationxml')
ordered_stations = [inv[0].stations[i].code for i in range(16)]
ordered_locations = [(float(inv[0].stations[i].extra.local_location.value['x'].value),
                      float(inv[0].stations[i].extra.local_location.value['y'].value),
                      float(inv[0].stations[i].extra.local_location.value['z'].value))
                     for i in range(16)]


These bits of extra info are great candidates for ASDF auxiliary data.

In [6]:
ordered_locations

[(27.45232, 19.05, 0.0),
 (41.447720000000004, 19.05, 0.0),
 (34.46272, 19.05, 0.0),
 (30.957520000000002, 19.05, 0.0),
 (19.558, 16.002, 0.0),
 (20.44192, 19.05, 0.0),
 (34.29, 26.162000000000003, 0.0),
 (37.846000000000004, 22.352000000000004, 0.0),
 (30.957520000000002, 22.555200000000003, 0.0),
 (26.46172, 23.368, 0.0),
 (23.876, 22.352000000000004, 0.0),
 (13.43152, 19.05, 0.0),
 (35.45332, 14.732, 0.0),
 (30.957520000000002, 15.5448, 0.0),
 (9.9568, 19.05, 0.0),
 (23.876, 15.748000000000001, 0.0)]

In [9]:
np.array([s.encode('utf8') for s in ordered_stations])
# h5 files can't handle the numpy unicode datatype, this is a workaround

array([b'AE27', b'AE09', b'AE17', b'AE22', b'AE38', b'AE35', b'AE18',
       b'AE11', b'AE21', b'AE28', b'AE32', b'AE41', b'AE16', b'AE23',
       b'AE43', b'AE33'], dtype='|S4')

In [7]:
ds.add_auxiliary_data(data=np.array(ordered_locations),
                      data_type='LabStationInfo',
                      path='local_locations',
                      parameters={})
ds.add_auxiliary_data(data=np.array([s.encode('utf8') for s in ordered_stations]),
                      data_type='LabStationInfo',
                      path='ordered_stations',
                      parameters={})

In [8]:
ds.auxiliary_data.LabStationInfo.local_locations.data[:]

array([[27.45232, 19.05   ,  0.     ],
       [41.44772, 19.05   ,  0.     ],
       [34.46272, 19.05   ,  0.     ],
       [30.95752, 19.05   ,  0.     ],
       [19.558  , 16.002  ,  0.     ],
       [20.44192, 19.05   ,  0.     ],
       [34.29   , 26.162  ,  0.     ],
       [37.846  , 22.352  ,  0.     ],
       [30.95752, 22.5552 ,  0.     ],
       [26.46172, 23.368  ,  0.     ],
       [23.876  , 22.352  ,  0.     ],
       [13.43152, 19.05   ,  0.     ],
       [35.45332, 14.732  ,  0.     ],
       [30.95752, 15.5448 ,  0.     ],
       [ 9.9568 , 19.05   ,  0.     ],
       [23.876  , 15.748  ,  0.     ]])

Now on to importing the waveforms. An experiment might be made up of multiple files. Display the tpc5 files present in the experiment folder:

In [3]:
!ls 020918_shear/*.tpc5

020918_shear/bd1.tpc5  020918_shear/run.tpc5
020918_shear/bd2.tpc5  020918_shear/timing.tpc5


This experiment had two ball drops (bd1, bd2), a timing alignment signal (timing), and the full shear run (run). Start here (after imports) to add each tpc5 file.

The timing file only has one channel (D4). This is under 00000001 in the tpc5 name hierarchy, so without any tpc5 channel name checking it gets dumped into A1 here. It was also moved, so it's not even really valid to put it in AE41. Maybe I can use AE99 as a dummy location.

In [14]:
f = h5py.File("020918_shear/run.tpc5", "r")

In [15]:
# we need to know how many blocks to read
# all channels have the same number of blocks, use channel 1
cg = f[tpc5.getChannelGroupName(1)]
nblocks = len(cg['blocks'].keys())

In [6]:
ordered_stations = ds.auxiliary_data.LabStationInfo.ordered_stations.data
ordered_stations = [stat.decode("utf-8") for stat in ordered_stations]
#ordered_stations

In [13]:
# SPECIAL block for timing signal at temporary off-plate station
# first build some basic Stats
statn_stats = Stats()
statn_stats.network = 'L0'
statn_stats.channel = 'FHZ'
statn_stats.location = '00'
statn_stats.sampling_rate = 20e6

# make sure times will retain full precision
# the ElSys max precision seems to be nanoseconds
obspy.UTCDateTime.DEFAULT_PRECISION = 9

statn_stats.station = 'AE99'
statn_stream = Stream()

statn_stats.starttime = (obspy.UTCDateTime(tpc5.getStartTime(f,1)) # gives the start of the whole recording
                         + tpc5.getTriggerTime(f,1,block=blk) # seconds from start to trigger
                         - tpc5.getTriggerSample(f,1,block=blk)/statn_stats.sampling_rate) # seconds from trigger to block start
        
# get the raw voltage data
raw = tpc5.getVoltageData(f, 1)
# give the stats the length, otherwise it takes 0 points
statn_stats.npts = len(raw)
statn_stream += Trace(raw ,statn_stats)
    
# add the complete stream to the ASDF object
ds.add_waveforms(statn_stream, "timing")

In [16]:
# read the raw data from each channel into a stream, one trace at a time
# first build some basic Stats
statn_stats = Stats()
statn_stats.network = 'L0'
statn_stats.channel = 'FHZ'
statn_stats.location = '00'
statn_stats.sampling_rate = 20e6

# make sure times will retain full precision
# the ElSys max precision seems to be nanoseconds
obspy.UTCDateTime.DEFAULT_PRECISION = 9

# iterate through stations, following the ordered_stations A1-D4 sort
# the number of the (ordered) station is the TranAX channel A1-D4->1-16, here Tchan
for Tchan,statname in enumerate(ordered_stations,1):
    # add the station to the stats
    statn_stats.station = statname
    print(statname)
    
    # create a stream for the station
    statn_stream = Stream()
    
    # iterate through continuous data segments
    # TranAX calls these Blocks, obspy calls them Traces
    for blk in range(1,nblocks+1):
        # get the trace start time
        statn_stats.starttime = (obspy.UTCDateTime(tpc5.getStartTime(f,1)) # gives the start of the whole recording
                                + tpc5.getTriggerTime(f,1,block=blk) # seconds from start to trigger
                                - tpc5.getTriggerSample(f,1,block=blk)/statn_stats.sampling_rate) # seconds from trigger to block start
        
        # get the raw voltage data
        raw = tpc5.getVoltageData(f, Tchan, block=blk)
        # give the stats the length, otherwise it takes 0 points
        statn_stats.npts = len(raw)
        statn_stream += Trace(raw ,statn_stats)
    
    # add the complete stream to the ASDF object
    ds.add_waveforms(statn_stream, "run")

AE27
AE09
AE17
AE22
AE38
AE35
AE18
AE11
AE21
AE28
AE32
AE41
AE16
AE23
AE43
AE33


In [17]:
ds.waveforms.L0_AE09.list()

['L0.AE09.00.FHZ__2018-02-09T22:15:02.828951__2018-02-09T22:15:02.828951__bd1',
 'L0.AE09.00.FHZ__2018-02-09T22:17:49.024309__2018-02-09T22:17:49.024309__bd2',
 'L0.AE09.00.FHZ__2018-02-09T22:30:51.423920__2018-02-09T22:30:51.423920__run',
 'L0.AE09.00.FHZ__2018-02-09T22:30:59.677762__2018-02-09T22:30:59.677762__run',
 'L0.AE09.00.FHZ__2018-02-09T22:31:33.355811__2018-02-09T22:31:33.355811__run',
 'L0.AE09.00.FHZ__2018-02-09T22:32:38.097585__2018-02-09T22:32:38.097585__run',
 'L0.AE09.00.FHZ__2018-02-09T22:36:00.631816__2018-02-09T22:36:00.631816__run',
 'L0.AE09.00.FHZ__2018-02-09T22:36:00.976014__2018-02-09T22:36:00.976014__run',
 'L0.AE09.00.FHZ__2018-02-09T22:36:00.999154__2018-02-09T22:36:00.999154__run',
 'L0.AE09.00.FHZ__2018-02-09T22:36:01.172681__2018-02-09T22:36:01.172681__run',
 'L0.AE09.00.FHZ__2018-02-09T22:36:01.232056__2018-02-09T22:36:01.232056__run',
 'L0.AE09.00.FHZ__2018-02-09T22:36:01.252013__2018-02-09T22:36:01.252013__run',
 'L0.AE09.00.FHZ__2018-02-09T22:36:01.28

In [114]:
ds.waveforms.L0_AE27.raw_record[0].stats

         network: L0
         station: AE27
        location: 00
         channel: FHZ
       starttime: 2018-02-09T22:30:51.423920384Z
         endtime: 2018-02-09T22:30:51.423920384Z
   sampling_rate: 20000000.0
           delta: 5e-08
            npts: 0
           calib: 1.0
         _format: ASDF
            asdf: AttribDict({'format_version': '1.0.1', 'tag': 'raw_record'})

## Add TDMS data
The package npTDMS has support for DAQmx raw data, so it's very easy to add here as auxiliary data.

In [19]:
tdms = TdmsFile('020918_shear/2.9.2018 - 2.25.37 PM/Board_5_25000_Hz.tdms')
shear_volts = tdms.object('record','PXI1Slot6/ai7').data
normal_volts = tdms.object('record','PXI1Slot6/ai6').data
load_start = obspy.UTCDateTime(tdms.object('record','PXI1Slot6/ai6').property('wf_start_time'))


In [20]:
ds.add_auxiliary_data(data=shear_volts,
                      data_type='LoadData',
                      path='shear_volts',
                      parameters={})

In [21]:
ds.add_auxiliary_data(data=normal_volts,
                      data_type='LoadData',
                      path='normal_volts',
                      parameters={})

In [24]:
ds.flush()
ds._close()

## Extra notes
Accessing tpc5 and hdf5 files can be a bit confusing. Here are some reminders:

In [ ]:
# to view the keys of an hdf5 file:
list(f.keys())

# to then access one of the keys:
f['key']

# to control the precision of a UTCDateTime
t = obspy.UTCDateTime(precision=9)
# doing arithmetic creates a new UTCDateTime with the default precision!
# change the default precision for the session
obspy.UTCDateTime.DEFAULT_PRECISION = 9

In [18]:
# clean up a botched attempt at adding raw_records
[ds.waveforms['L0_'+code].__delitem__('timing') for code in ordered_stations]

WaveformNotInFileException: Tag 'timing' not part of the data set for station 'L0.AE09'.